In [1]:
import json
from shapely.geometry import shape, MultiPolygon, Polygon
from shapely.ops import unary_union
import pandas as pd
import json
from shapely.geometry import Point, Polygon, MultiPolygon
import pandas as pd
# from shapely.geometry import shape,Point, Polygon, MultiPolygon
from shapely.ops import transform
from pyproj import Transformer
import json  
from geopy.distance import geodesic 
from tqdm import tqdm 
import pandas as pd 
import numpy as np

In [2]:
adm2_data_pd = pd.read_excel('input_data/economy_attractions.xlsx')

adm2_data = adm2_data_pd.to_dict(orient='records')
distances = {}

for i, place1 in tqdm(enumerate(adm2_data), total=len(adm2_data)):
    place1_key = f"{place1['id']}"
    distances[place1_key] = {}
    for j, place2 in enumerate(adm2_data[i+1:]):
        place2_key = f"{place2['id']}"
        dist = geodesic((place1['Latitude'], place1['Longitude']),
                        (place2['Latitude'], place2['Longitude'])).km
        distances[place1_key][place2_key] = dist
        
with open('data/adm2_distances.txt', 'w') as f:
    for place1, dist_dict in distances.items():
        for place2, dist in dist_dict.items():
            f.write(f"{place1}\t{place2}\t{dist:.2f}\n")

100%|██████████| 297/297 [00:03<00:00, 80.14it/s] 


In [3]:
transportation_data_pd = pd.read_excel('input_data/transportation.xlsx')

adm2_data_pd = pd.merge(
    adm2_data_pd,
    transportation_data_pd[['id', 'Has Train Station', 'Has Airport']],
    on='id',
    how='left'
)
adm2_data_pd.to_excel('data/transportation_economy_attractions.xlsx', index=False)

In [4]:
adm2_transportation_economy_attractions=pd.read_excel('data/transportation_economy_attractions.xlsx')
adm2_transportation_economy_attractions['id'] = adm2_transportation_economy_attractions['id'].astype(str)
adm2_area = dict(zip(adm2_transportation_economy_attractions['id'], adm2_transportation_economy_attractions['Area']))
# adm2_transportation_economy_attractions[['total_attractions', 'total_reviews', 'five_score', 'four_score', 'three_score', 'two_score', 'one_score']] = adm2_transportation_economy_attractions[['total_attractions', 'total_reviews', 'five_score', 'four_score', 'three_score', 'two_score', 'one_score']].fillna(0)
adm2_transportation_economy_attractions['tourism_quality'] = (
    10 * adm2_transportation_economy_attractions['5A'] +
    6 * adm2_transportation_economy_attractions['4A'] +
    2 * adm2_transportation_economy_attractions['3A']
)

adm2_transportation_economy_attractions['Population'] = adm2_transportation_economy_attractions['Population'].replace(',', '', regex=True).astype(float)
adm2_transportation_economy_attractions['GDP'] = adm2_transportation_economy_attractions['GDP'].replace(',', '', regex=True).astype(float)

adm2_population = dict(zip(adm2_transportation_economy_attractions['id'], adm2_transportation_economy_attractions['Population']))
adm2_income = dict(zip(adm2_transportation_economy_attractions['id'], adm2_transportation_economy_attractions['GDP']))
adm2_tourism_quality = dict(zip(adm2_transportation_economy_attractions['id'], adm2_transportation_economy_attractions['tourism_quality']))
adm2_transportation_economy_attractions['id'] = adm2_transportation_economy_attractions['id'].astype(int)
        
adm2_distances = {}
with open('data/adm2_distances.txt', 'r') as f:
    for line in f:
        place1, place2, dist = line.strip().split('\t')
        dist = float(dist)
        if place1 not in adm2_distances:
            adm2_distances[place1] = {}
        if place2 not in adm2_distances:
            adm2_distances[place2] = {}
        adm2_distances[place1][place2] = dist
        adm2_distances[place2][place1] = dist

In [5]:
with open('data/population_attraction_scores.txt', 'w') as f:
    for place1, dist_dict in adm2_distances.items():
        if place1 in adm2_population:
            pop1 = adm2_population[place1]
            total_score = 0
            for place2, dist in dist_dict.items():
                if place2 in adm2_population and dist > 0:
                    pop2 = adm2_population[place2]
                    score = (pop1 * pop2) / dist
                    total_score += score
            f.write(f"{place1}\t{total_score:.2f}\n")
            
with open('data/tourism_attraction_scores.txt', 'w') as f:
    for place1, dist_dict in adm2_distances.items():
        if place1 in adm2_tourism_quality:
            pop1 = adm2_tourism_quality[place1]
            total_score = 0
            for place2, dist in dist_dict.items():
                if place2 in adm2_tourism_quality and dist > 0:
                    pop2 = adm2_tourism_quality[place2]
                    score = (pop1 * pop2) / dist
                    total_score += score
            f.write(f"{place1}\t{total_score:.2f}\n")
            
            
def self_potential(GDP_i, area_i):
    return GDP_i / ((2 / 3) * np.sqrt(area_i / np.pi))


HMP = {}
for place1 in adm2_income:
    GDP_i = adm2_income[place1]
    area_i = adm2_area[place1]

    potential_sum = 0
    for place2, dist in adm2_distances.get(place1, {}).items():
        if place1 != place2:
            GDP_j = adm2_income[place2]
            potential_sum += GDP_j / dist
    
    self_term = self_potential(GDP_i, area_i)
    
    HMP[place1] = potential_sum + self_term

HMP_df = pd.DataFrame(list(HMP.items()), columns=['adm_code', 'HMP'])
HMP_df.to_csv('data/harris_market_potential.txt', sep='\t', index=False, header=False)

In [6]:
population_attraction_scores = pd.read_csv('data/population_attraction_scores.txt', sep='\t', header=None, names=['id', 'population_attraction_score'])
tourism_attraction_scores = pd.read_csv('data/tourism_attraction_scores.txt', sep='\t', header=None, names=['id', 'tourism_attraction_scores'])
harris_market_potential = pd.read_csv('data/harris_market_potential.txt', sep='\t', header=None, names=['id', 'harris_market_potential'])

merged_data = pd.merge(adm2_transportation_economy_attractions, population_attraction_scores, on='id', how='left')
merged_data = pd.merge(merged_data, tourism_attraction_scores, on='id', how='left')
merged_data = pd.merge(merged_data, harris_market_potential , on='id', how='left')

merged_data = merged_data.rename(columns={
    'city_name': 'ADM2',
    'id': 'id'
})
merged_data.to_excel('data/final_scores.xlsx', index=False)